# Before you start:
### Make sure you run this in the DEEPLABCUT virtual environment

In [1]:
#imports and definitions
import pandas as pd
import deeplabcut
import os
def find_deeplabcut_project(_base_dir, _project_name, _experimenter_name):
    for folder in os.listdir(_base_dir):
        if folder.startswith(_project_name + "-" + _experimenter_name):
            return os.path.join(_base_dir, folder)
    return None

Loading DLC 3.0.0rc9...


# Select project paths

In [2]:
#Run this always, edit the 2 variables below
project_name = "oscillator" #IMPORTANT: NO SPACES
experimenter_name = "Nils-Sommerfeld" #IMPORTANT: NO SPACES


#do not edit
base_directory = os.getcwd() #The new folder you created
if " " in project_name:
    raise ValueError("Space in project_name")
if " " in experimenter_name:
    raise ValueError("Space in experimenter_name")

# Create new project
Set any image from your training data to pass

In [4]:
#enter the path of any of your training data images (below is assumed that your training data is in your project folder, otherwise
#    edit the 3rd argument in deeplabcut.create_new_project()
training_image = "training_data_750\\osc0001"


training_image_extension = ".png"
deeplabcut.create_new_project(
    project_name,
    experimenter_name,
    [base_directory + "\\" + training_image + training_image_extension],
    copy_videos = False,
    multianimal = False,
)

Project "C:\Users\somme\Documents\DNA\pose_test\oscillator-Nils-Sommerfeld-2025-07-25" already exists!


'C:\\Users\\somme\\Documents\\DNA\\pose_test\\oscillator-Nils-Sommerfeld-2025-07-25\\config.yaml'

# Configure project location
### Before you continue, do 2 things.
### First, add bodyparts. Go into the config.yaml file (which exists in the project folderthat was just created) and manually add/edit the bodyparts of your project
### Second, paste your training data into the folder (which should share names with the image you passed when you created the project) and is in the 'labeled-data' folder in the project that was just created

In [3]:
config_path = find_deeplabcut_project(base_directory, project_name, experimenter_name) + "\\config.yaml"
print (config_path)

C:\Users\somme\Documents\DNA\pose_test\oscillator-Nils-Sommerfeld-2025-07-25\config.yaml


# Label Data

In [4]:
deeplabcut.label_frames(config_path)
#If asked for a version of the napari reader, the first one works fine, I don't know if the second one works.
#You must open the config file for your project with open file
#You must open the folder you want to use as your training data with open folder (this folder should be under labeled-data in the project folder)
#After you are done labelling, make sure you select the layer called "CollectedData" or something to that degree,
#    then do "Save Selected Layer(s)" or ctrl+s, and an h5 and csv file should appear in the folder with the images.
#This may throw an error if you've done something wrong

#Also if anything seems weird with your layers, you might have more than 2 open, you only want one layer of images and one layer of collected data.
#If you do have more that 2 open, just delete them all and reopen your config file and folder.

## Optional: Create images of labeled frames

In [39]:
deeplabcut.check_labels(config_path, visualizeindividuals=True)
#This creates an additional folder for each "video file" in the labeled-data folder which has images of all labeled frames

Creating images with labels by Nils-Sommerfeld.


100%|██████████| 15/15 [00:00<00:00, 85.66it/s]

If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


# Train Network

In [10]:
#WARNING: This will create a dataset that automatically uses 5% of your labeled frames as validation data.
# If you want to change how much of your data is validation vs. training, you can edit that in the config.yaml file/
temp = deeplabcut.create_training_dataset(config_path)
print(temp)

C:\Users\somme\Documents\DNA\pose_test\oscillator_test-Nils-Sommerfeld-2025-07-24\labeled-data\test_validate\CollectedData_Nils-Sommerfeld.h5  not found (perhaps not annotated).
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!
[(0.95, 1, (array([ 2, 28, 13, 10, 26, 24, 27, 11, 17, 22,  5, 16,  8, 14, 23, 20,  1,
       29,  6,  4, 18, 19,  9,  7, 25,  3,  0, 21]), array([15, 12])))]


In [ ]:
deeplabcut.train_network(
    config_path,
    shuffle=1,
    trainingsetindex=0,
    gputouse=0,                   # instead of device="cuda:0"
    max_snapshots_to_keep=5,
    displayiters=100,
    saveiters=5,                  # note: saveiters, not save_epochs
    maxiters=20,             # epochs aren't a thing here—use iterations
    allow_growth=True
)
#This is going to give a big red screen but it will be fine
#This may also take a while. It may be worth it instead of running this on your own machine to run it in a google co-lab or with something else
#Also there is some functionality that saves training checkpoints after a certain number of epochs, and then training can be continued from there.

Config:
{'all_joints': [[0], [1], [2], [3], [4], [5], [6], [7], [8]],
 'all_joints_names': ['long_plat',
                      'long_hinge',
                      'long_arm',
                      'inner_arm_long',
                      'inner_hinge',
                      'inner_arm_short',
                      'short_arm',
                      'short_hinge',
                      'short_plat'],
 'alpha_r': 0.02,
 'apply_prob': 0.5,
 'batch_size': 1,
 'contrast': {'clahe': True,
              'claheratio': 0.1,
              'histeq': True,
              'histeqratio': 0.1},
 'convolution': {'edge': False,
                 'emboss': {'alpha': [0.0, 1.0], 'strength': [0.5, 1.5]},
                 'embossratio': 0.1,
                 'sharpen': False,
                 'sharpenratio': 0.3},
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_oscillator_testJul24\\oscillator_test_Nils-Sommerfeld95shuffle1.mat',
 'dataset_type': 'default',
 

Selecting single-animal trainer
Batch Size is 1


C:\Users\somme\anaconda3\envs\DEEPLABCUT\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Loading ImageNet-pretrained resnet_50


C:\Users\somme\anaconda3\envs\DEEPLABCUT\lib\site-packages\deeplabcut\pose_estimation_tensorflow\core\train.py:228: UserWarning: Switching to Adam, as SGD crashes on Apple Silicon.
  warnings.warn("Switching to Adam, as SGD crashes on Apple Silicon.")
Exception in thread Thread-5 (load_and_enqueue):
Traceback (most recent call last):
  File "C:\Users\somme\anaconda3\envs\DEEPLABCUT\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\somme\anaconda3\envs\DEEPLABCUT\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\somme\anaconda3\envs\DEEPLABCUT\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\somme\anaconda3\envs\DEEPLABCUT\lib\site-packages\deeplabcut\pose_estimation_tensorflow\core\train.py", line 84, in load_and_enqueue
    batch_np = dataset.next_batch()
  File "C:\Users\somme\anaconda3\envs\DEEPLABCUT\lib\site-packages\deeplabcut\pose_es

Max_iters overwritten as 20
Display_iters overwritten as 100
Save_iters overwritten as 5
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': 'C:\\Users\\somme\\Documents\\DNA\\pose_test\\oscillator_test-Nils-Sommerfeld-2025-07-24\\dlc-models\\iteration-0\\oscillator_testJul24-trainset95shuffle1\\train\\snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'adam', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 1, 'dataset_type': 'default', 'deterministic': False, 'mirror': False, 'pairwise_huber_loss': False, 'weigh_only_present_joints': False, 'partaffinityfield_predict': False, 'pairwise_predict': False, 'all

# Refine
Now you can move on to deeplabcut_refine.ipynb